In [6]:

import transformers

In [1]:
from vllm import LLM, SamplingParams

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
#model_id="meta-llama/Meta-Llama-3.1-8B-Instruct"

tensor_parallel_size = 8
# トークナイザーとモデルの準備
model = LLM(
    model=model_id,
    trust_remote_code=True,
    max_model_len=2000,
    tensor_parallel_size=tensor_parallel_size,
)

INFO 09-17 17:31:41 config.py:904] Defaulting to use mp for distributed inference
INFO 09-17 17:31:41 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llam

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


(VllmWorkerProcess pid=543385) INFO 09-17 17:31:53 weight_utils.py:242] Using model weights format ['*.safetensors']
INFO 09-17 17:32:00 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543391) INFO 09-17 17:32:00 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543389) INFO 09-17 17:32:01 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543390) INFO 09-17 17:32:01 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543388) INFO 09-17 17:32:01 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543386) INFO 09-17 17:32:02 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543385) INFO 09-17 17:32:03 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543387) INFO 09-17 17:32:03 model_runner.py:1008] Loading model weights took 16.4605 GB
INFO 09-17 17:32:07 distri

In [2]:
temperature=0.7
top_k=50

In [13]:
def llm_gen(model,prompt_list,temperature=0.7,top_k=50):
    outputs = model.generate(
        prompt_list,
        sampling_params=SamplingParams(
            temperature=temperature,
            max_tokens=1024,
            #repetition_penalty=1.2,
            top_k=top_k,
        )
    )
    return [i.outputs[0].text.strip() for i in outputs]

In [ ]:

tokenizer=transformers.AutoTokenizer.from_pretrained(model_id)

In [24]:
from datasets import load_dataset

ds=load_dataset("kanhatakeyama/material-properties",split="train")
ds=ds.shuffle()

{'CompName': 'unknown',
 'SMILES': 'C1=CC(=CC(=C1)CN)CN',
 'Property': 'Boiling temperature',
 'Value': 247.2222222,
 'unit': '[oC]',
 'Source': 'Wikipedia/Wikidata'}

In [45]:


def gen_problem(record):
    comp_name=record["CompName"]
    smiles=record["SMILES"]
    unit=record["unit"]
    property_name=record["Property"]
    q=f"""Predict the {property_name} {unit} of the following compound. 
    #Restriction: Output must contain "#Reason" section which explains the step-by-step quantitative reasoning behind the prediction. Output must also contain "#Predicted" section which contains only the predicted value (number only).
    #Name: {comp_name}
    #SMILES: {smiles}"""
    actual_value=record["Value"]

    chat = [
        {"role": "user", "content": q},
    ]


    prompt=tokenizer.apply_chat_template(chat,tokenize=False,)
    assist_prefix="assistant\n\n#Reason\n"
    prompt+=assist_prefix

    return prompt,actual_value

In [ ]:
n_records=10

In [70]:
import random
problems=[]
prompt_list=[
]
n_records=20
for i in range(n_records):
    record=random.choice(ds)
    prompt,actual_value=gen_problem(record)
    problems.append(
        {
         "record":record,
         "prompt":prompt,
         "actual_value":actual_value   
        }

    )
    prompt_list.append(prompt)
problems

[{'record': {'CompName': 'unknown',
   'SMILES': 'CC1=C(C=C(C=C1[N+](=O)[O-])[N+](=O)[O-])[N+](=O)[O-]',
   'Property': 'Melting temperature',
   'Value': 80.0,
   'unit': '[oC]',
   'Source': 'Wikipedia/Wikidata'},
  'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPredict the Melting temperature [oC] of the following compound. \n    #Restriction: Output must contain "#Reason" section which explains the step-by-step quantitative reasoning behind the prediction. Output must also contain "#Predicted" section which contains only the predicted value (number only).\n    #Name: unknown\n    #SMILES: CC1=C(C=C(C=C1[N+](=O)[O-])[N+](=O)[O-])[N+](=O)[O-]<|eot_id|>assistant\n\n#Reason\n',
  'actual_value': 80.0},
 {'record': {'CompName': 'Propyl gallate',
   'SMILES': 'O=C(OCCC)c1cc(O)c(O)c(O)c1',
   'Property': 'Melting temperature',
   'Value': 150.0

In [71]:
predicted_text_list=llm_gen(model,prompt_list)

Processed prompts:   0%|          | 0/20 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 20/20 [00:16<00:00,  1.21it/s, est. speed input: 150.48 toks/s, output: 369.16 toks/s]


In [72]:
for i in range(n_records):
    problems[i]["CompoundName"]=problems[i]["record"]["CompName"]
    problems[i]["SMILES"]=problems[i]["record"]["SMILES"]
    problems[i]["Property"]=problems[i]["record"]["Property"]
    problems[i]["Unit"]=problems[i]["record"]["unit"]
    problems[i]["predicted"]=predicted_text_list[i].split("#Predicted\n")[-1]
    problems[i]["predicted_text"]=predicted_text_list[i]

In [76]:
import pandas as pd
df=pd.DataFrame(problems)
df=df.drop(columns=["record","prompt"])
df.to_csv("predicted.csv",index=False)

In [77]:
df

,actual_value,CompoundName,SMILES,Property,Unit,predicted,predicted_text
0,80.000000,unknown,CC1=C(C=C(C=C1[N+](=O)[O-])[N+](=O)[O-])[N+](=...,Melting temperature,[oC],150.50,To predict the melting temperature of the give...
1,150.000000,Propyl gallate,O=C(OCCC)c1cc(O)c(O)c(O)c1,Melting temperature,[oC],204.9,The melting point of a compound can be predict...
2,207.716250,unknown,CCCC,Vapor pressure,[kPa],2.41,To predict the vapor pressure of the given com...
3,-61.300000,Bromine pentafluoride,FBr(F)(F)(F)F,Melting temperature,[oC],29.411,To predict the melting temperature of Bromine ...
4,1.030000,N-Methyltyramine,Oc1ccc(cc1)CCNC,Density,[g/cm3],0.96,"To predict the density of N-Methyltyramine, we..."
5,-72.222222,unknown,C(C#N)O,Melting temperature,[oC],210,To predict the melting temperature of the give...
6,707.000000,Arsenic trisulfide,S1[As]3S[As]2S[As](S[As]1S2)S3,Boiling temperature,[oC],261,The boiling temperature of a compound can be p...
7,253.000000,unknown,CCOC1=CC2=C(C=C1)[N+](=C(C=C2)C=CC3=CC(=CC=C3)...,Melting temperature,[oC],180,The given compound is a complex organic molecu...
8,1.080000,Ergosterol peroxide,O[C@@H]2C[C@]\13OO[C@]4(/C=C/1)[C@@H]([C@]3(CC...,Density,[g/cm3],1.0,"To predict the density of Ergosterol peroxide,..."
9,51.111111,unknown,CCOCCOC(=O)C,Flash temperature,[oC],55.6,The compound is a carbon-based molecule with a...
